In [ ]:
import pydeck as pdk

from datetime import datetime
import pandas as pd

# Data from setlist.fm. See website for further information.
df = pd.read_csv('tours.csv')
df.tail()

In [ ]:
import random
import ipywidgets

# Base map
COUNTRIES = "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_admin_0_scale_rank.geojson"

# Use the experimental GlobeView
view = pdk.View(type="_GlobeView", controller=True, width=1000, height=700)

# Data Cleaning
df = df.fillna('')
color_lookup = pdk.data_utils.assign_random_colors(df['tour_name'])
df['c'] = df['tour_name'].apply(lambda row: color_lookup.get(row))
df['random'] = df['tour_name'].apply(lambda row: random.random() * 10)

layers = [
    pdk.Layer(
        "GeoJsonLayer",
        id="base-map",
        data=COUNTRIES,
        stroked=False,
        filled=True,
        get_line_color=[60, 60, 60],
        get_fill_color=[200, 200, 200],
    ),
    pdk.Layer(
        "ArcLayer",
        id="artist_map",
        data=df,
        get_source_position=["lng", "lat"],
        get_target_position=["lng_lagged", "lat_lagged"],
        get_source_color='c',
        get_target_color='c',
        opacity=0.6,
        get_tilt='15 * random',
        pickable=True,
        auto_highlight=True,
    ),
    pdk.Layer(
        "ScatterplotLayer",
        id="artist_map_points",
        data=df,
        get_position=["lng", "lat"],
        get_fill_color='c',
        radius_min_pixels=5,
        opacity=0.6,
        get_radius=300,
        pickable=True,
        auto_highlight=True,
    ),
]

view_state = pdk.ViewState(latitude=40.714269, longitude=-74.005973, zoom=8)

deck = pdk.Deck(
    views=[view],
    initial_view_state=view_state,
    tooltip={"text": "{tour_name}, {artist}"},
    layers=layers,
    map_provider=None,
    # Note that this must be set for the globe to be opaque
    parameters={"cull": True},
)

selector = ipywidgets.Dropdown(
    options=list(set(df.artist)),
    value='Bob Dylan',
    description='Artist:',
    disabled=False,
)


def render():
    global deck
    global selector
    display(selector)
    deck.layers[1].data = df[df['artist'] == 'Bob Dylan']
    deck.layers[2].data = df[df['artist'] == 'Bob Dylan']
    deck.update()
    display(deck.show())
    
def on_selection(change):
    global deck
    global df

    deck.layers[1].data = df[df['artist'] == change['new']]
    deck.layers[2].data = df[df['artist'] == change['new']]
    deck.update()

selector.observe(on_selection, names='value')

render()